# Tutorial 3 - Memory

## Key Philosophy
- It would be important to learn from past experience and improve the agentic framework - memory is key to that
- You can add to the memory bank of your Agents pre-inference (by collecting from a pool of data prior to running the Agent), or during inference (add on in between running subtasks)

- In general, for the various Memory classes, you use:
    - `append(list_of_memories)`: append a list of memories to the memory bank
    - `retrieve(task: str)`: retrieves top_k memories based on task 
    
## Use Memory in Agents
- Agent class takes `memory_bank` as a parameter during initialisation of an `Agent`
- memory_bank: class Dict[Memory]. Stores multiple types of memory for use by the agent. Customise the Memory config within the Memory class.

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [4]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [5]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Memory Class 1: Base Memory Class

- In-house Vector DB that can store anything in a Python list for retrieval to augment the Agent's prompt
- Retrieves top k memory items based on task 
- Inputs:
    - `memory`: List. Default: Empty List. The list containing the memory items
    - `top_k`: Int. Default: 5. The number of memory list items to retrieve
    - `mapper`: Function. Maps the memory item to another form for comparison by ranker or LLM. Default: `lambda x: x`
        - Example mapping: `lambda x: x.fn_description` (If x is a Class and the string you want to compare for similarity is the fn_description attribute of that class)
    - `approach`: str. Either `retrieve_by_ranker` or `retrieve_by_llm` to retrieve memory items.
        - Ranker is faster and cheaper as it compares via embeddings, but are inferior to LLM-based methods for contextual information
    - `retrieve_fn`: Default: None. Takes in task and outputs top_k similar memories in a list
    - `ranker`: `Ranker`. The Ranker which defines a similarity score between a query and a key. Default: OpenAI `text-embedding-3-small` model. 
        - Can be replaced with a function which returns similarity score from 0 to 1 when given a query and key
        
## In-built Memory in TaskGen: Function Memory
- Default: `memory_bank = {'Function': Memory(top_k = 5, mapper = lambda x: x.fn_description, approach = 'retrieve_by_ranker'), llm = self.llm}`
- Does RAG over Task -> Function mapping

## Use Case 1: Filtering Functions by Task
- TaskGen chooses `top k` (default k: 5) functions according to similarity to subtask
- In addition to `top k` functions, we will also give agent all the compulsory functions
    - `is_compulsory` variable of Function set to `True` means that we will always have it as one of the functions for planning and bypass Function RAG
    
## Example Use Case
- Helps to reduce number of functions present in LLM context for more accurate generation
```python
output = my_agent.run('Calculate 2**10 * (5 + 1) / 10')
```
`Original Function List: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, divide_numbers, power_of, GCD_of_two_numbers, modulo_of_numbers, absolute_difference, generate_poem_with_numbers, List_related_words, generate_quote`

`Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power_of, modulo_of_numbers`


In [6]:
from typing import List
import math
def sum_numbers(num_list: List[float]) -> float:
    '''Adds all numbers in num_list'''
    return sum(x for x in num_list)

def subtract_numbers(num1: float, num2: float) -> float:
    '''Subtracts num1 from num2'''
    return num1 - num2

def multiply_numbers(num1: float, num2: float) -> float:
    '''Multiplies num1 by num2'''
    return num1 * num2

def divide_numbers(num1: float, num2: float) -> float:
    '''Divides num1 by num2'''
    if num2 == 0:
        return -1
    return num1/num2

def power_operation(num1: float, num2: float) -> float:
    '''Returns num1 to the power of num2 (num1**num2)'''
    return math.pow(num1, num2)

def greatest_common_divisor(num1: int, num2: int) -> int:
    '''Returns greatest common divisor of num1 and num2'''
    return math.gcd(num1, num2)

def modulo(num1: int, num2: int) -> int:
    '''Returns modulo of num1 over num2'''
    return num1%num2

def absolute_difference(num1: int, num2: int) -> int:
    '''Returns absolute difference between num1 and num2'''
    return math.abs(num1-num2)

# Put this to make sum_numbers always appear for any task and bypass Function RAG
sum_numbers = Function(external_fn = sum_numbers, is_compulsory = True)

# This is for Internal Functions
generate_poem_with_numbers = Function("Generates a poem containing <num1: float> and <num2: float>", output_format = {"Poem": "Poem"}, fn_name = 'generate_poem_with_numbers', llm = llm)
list_related_words = Function("Lists out <num: int> words related to <word: str>", output_format = {"List of words": "List of words, type: list"}, fn_name = 'list_related_words', llm = llm)
generate_quote = Function("Generates a quote about <topic: str>", output_format = {"Quote": "Quote"}, fn_name = 'generate_quote', llm = llm)

In [7]:
my_agent = Agent('Generalist Agent', 
'''Does everything''',
                default_to_llm = False,
                llm = llm).assign_functions([sum_numbers, subtract_numbers, multiply_numbers, 
            divide_numbers, power_operation, greatest_common_divisor, modulo, absolute_difference, 
            generate_poem_with_numbers, list_related_words, generate_quote])

In [9]:
# see the auto-generated names of your functions :)
my_agent.print_functions()

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: sum_numbers
Description: Adds all numbers in <num_list: list[float]>
Input: ['num_list']
Output: {'output_1': 'float'}

Name: subtract_numbers
Description: Subtracts <num1: float> from <num2: float>
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: multiply_numbers
Description: Multiplies <num1: float> by <num2: float>
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: divide_numbers
Description: Divides <num1: float> by <num2: float>
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: power_operation
Description: Returns <num1: float> to the power of <num2: float> (<num1: float>**<num2: float>)
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: greatest_common_divisor
Description: Returns greatest common divisor of <num1: int> and <num2: int>
Input: ['num1', 'num2']
Output: {'output_1': 'int'}

Name: modulo
Description: Returns modulo of <num1: int> over 

In [10]:
# Configure your top_k for function filtering here, default is 5
my_agent.memory_bank['Function'].top_k = 5

In [12]:
# visualise how the Functions are chosen based on task - here you see subtract_numbers appearing at the front
# this does not include the compulsory functions
[f.fn_name for f in my_agent.memory_bank['Function'].retrieve('Evaluate 3 - 1')]

['modulo',
 'subtract_numbers',
 'multiply_numbers',
 'divide_numbers',
 'absolute_difference']

In [13]:
my_agent.reset()
my_agent.run('Evaluate 2+3')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: No subtasks have been completed yet for the assigned task of evaluating 2+3.
Thoughts: To complete the assigned task, I need to perform the addition of the numbers 2 and 3.
Subtask identified: Add the numbers 2 and 3 using the sum_numbers function.
Calling function sum_numbers with parameters {'num_list': [2.0, 3.0]}
> {'output_1': 5.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The sum of 2 and 3 has been successfully calculated, resulting in 5.0.
Thoughts: Since the task is to evaluate 2 + 3 and we have already computed the sum, the next step is to finalize the output and present it to the user.
Subtask identified: End Task
Task completed successfully!



[{'output_1': 5.0}]

In [14]:
my_agent.reset()
output = my_agent.run('Evaluate 2**10 * (3+5) / 10')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: No subtasks have been completed yet for the assigned task of evaluating the expression 2**10 * (3+5) / 10.
Thoughts: To complete the assigned task, I need to break down the expression into manageable parts. First, I can calculate the exponentiation 2**10, then evaluate the sum (3+5), and finally perform the multiplication and division.
Subtask identified: Calculate 2 raised to the power of 10.
Calling function power_operation with parameters {'num1': 2.0, 'num2': 10.0}
> {'output_1': 1024.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The power operation has been completed, calculating 2 raised to the power of 10, resulting in 1024.0.
Thoughts: Next, I need to evaluate the expression (3 + 5) to get the sum, and then multiply that result by 1024.0. After that,

In [15]:
my_agent.thoughts

[{'Observation': 'No subtasks have been completed yet for the assigned task of evaluating the expression 2**10 * (3+5) / 10.',
  'Thoughts': 'To complete the assigned task, I need to break down the expression into manageable parts. First, I can calculate the exponentiation 2**10, then evaluate the sum (3+5), and finally perform the multiplication and division.',
  'Current Subtask': 'Calculate 2 raised to the power of 10.',
  'Equipped Function Name': 'power_operation',
  'Equipped Function Inputs': {'num1': 2.0, 'num2': 10.0}},
 {'Observation': 'The power operation has been completed, calculating 2 raised to the power of 10, resulting in 1024.0.',
  'Thoughts': 'Next, I need to evaluate the expression (3 + 5) to get the sum, and then multiply that result by 1024.0. After that, I will divide the product by 10 to complete the assigned task.',
  'Current Subtask': 'Calculate the sum of 3 and 5.',
  'Equipped Function Name': 'sum_numbers',
  'Equipped Function Inputs': {'num_list': [3.0, 

In [16]:
my_agent.reply_user()

To evaluate the expression 2**10 * (3+5) / 10, we can break it down using the subtasks completed. First, we know that 2**10 is equal to 1024.0, which was calculated in the subtask power_operation(num1=2.0, num2=10.0). Next, we need to calculate (3+5), which is equal to 8.0, as found in the subtask sum_numbers(num_list=[3.0, 5.0]). Now, we can multiply these two results: 1024.0 * 8.0, which gives us 8192.0, as calculated in the subtask multiply_numbers(num1=1024.0, num2=8.0). Finally, we divide this result by 10.0: 8192.0 / 10.0, which results in 819.2, as shown in the subtask divide_numbers(num1=8192.0, num2=10.0). Therefore, the final result of the expression 2**10 * (3+5) / 10 is 819.2.


'To evaluate the expression 2**10 * (3+5) / 10, we can break it down using the subtasks completed. First, we know that 2**10 is equal to 1024.0, which was calculated in the subtask power_operation(num1=2.0, num2=10.0). Next, we need to calculate (3+5), which is equal to 8.0, as found in the subtask sum_numbers(num_list=[3.0, 5.0]). Now, we can multiply these two results: 1024.0 * 8.0, which gives us 8192.0, as calculated in the subtask multiply_numbers(num1=1024.0, num2=8.0). Finally, we divide this result by 10.0: 8192.0 / 10.0, which results in 819.2, as shown in the subtask divide_numbers(num1=8192.0, num2=10.0). Therefore, the final result of the expression 2**10 * (3+5) / 10 is 819.2.'

In [17]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['end_task', 'sum_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power_operation', 'greatest_common_divisor', 'modulo', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote']
Shared Variables: ['agent']
Task: Evaluate 2**10 * (3+5) / 10
Subtasks Completed:
Subtask: power_operation(num1=2.0, num2=10.0)
{'output_1': 1024.0}

Subtask: sum_numbers(num_list=[3.0, 5.0])
{'output_1': 8.0}

Subtask: multiply_numbers(num1=1024.0, num2=8.0)
{'output_1': 8192.0}

Subtask: divide_numbers(num1=8192.0, num2=10.0)
{'output_1': 819.2}

Subtask: Evaluate 2**10 * (3+5) / 10
To evaluate the expression 2**10 * (3+5) / 10, we can break it down using the subtasks completed. First, we know that 2**10 is equal to 1024.0, which was calculated in the subtask power_operation(num1=2.0, num2=10.0). Next, we need to calculate (3+5), which is equal to 8.0, as found in the s

## Use Case 2: Adding more context based on task
- You can add task-dependent context to the memory_bank so that `top k` will be added to prompt based on task

### Using `memory_bank` for more context
- Here, we have a mapping of nonsense words to numbers
- Based on the subtask, we will augment the system prompt with relevant mappings to aid planning

In [18]:
# first append more context to the agent
my_agent = Agent('Poem Creator', 'Create a poem according to a name', llm = llm)
my_agent.memory_bank['Task Instructions'] = Memory(['For John, generate a poem about the sun', 
'For Mary, generate a poem about the wind', 
'For Peter, generate a poem about the sea'], # some task-specific instructions
            top_k = 1,  # choose top 1
            mapper = lambda x: x) # we compare with the task using only the first word, e.g. Azo, Boneti, Andkh

In [19]:
# If name is present in memory, can use this information (e.g. John -> sun)
my_agent.reset()
output = my_agent.run('John')

Observation: No subtasks have been completed yet for the task of creating a poem for John.
Thoughts: To complete the task, I need to generate a poem about the sun specifically for John.
Subtask identified: Generate a poem about the sun for John.
Getting LLM to perform the following task: Generate a poem about the sun for John.
> In the golden embrace of dawn, John awakens to the sun, a radiant orb that dances across the sky, casting warmth and light upon the world. The sun, a painter of the day, splashes hues of orange and pink, igniting the horizon with its fiery brush. As it climbs higher, it bathes the earth in a gentle glow, nurturing the flowers and whispering to the trees. Each ray a promise, each beam a song, the sun sings of life, of hope, and of endless possibilities. John, like the sun, shines brightly, illuminating the lives of those around him, a beacon of joy and warmth in a world that often craves light.

Observation: A poem about the sun for John has been successfully ge

In [20]:
# If the name is not in the memory, it will be interpreted according to the actual input
my_agent.reset()
output = my_agent.run('Isaac Newton')

Observation: No subtasks have been completed yet for the assigned task of creating a poem about Isaac Newton.
Thoughts: To complete the assigned task, I need to generate a poem that reflects the life, achievements, and contributions of Isaac Newton. I can use the general function to create a poem since there are no specific instructions provided for this name.
Subtask identified: Generate a poem about Isaac Newton using the equipped function.
Getting LLM to perform the following task: Generate a poem about Isaac Newton using the equipped function.
> In the realm of science, a mind so bright, Isaac Newton, a beacon of light. With a gaze to the heavens, he pondered the stars, Unraveling mysteries, near and far. The apple that fell, a tale we all know, Inspired the laws of motion, a brilliant show. Gravity’s pull, a force so profound, In every small action, his genius was found. Calculus born from his tireless quest, A tool for the thinkers, a gift to the rest. Optics unveiled through pri

## Use Case 3: Add memory directly using pdf, docx, csv, xls files
Adding memory elements one by one can be cumbersome, TaskGen memory can take filepath as input and it will split the text content inside the file path either using default splitter or user provided splitter.

- We currently support `pdf`, `docx`, `csv`, `xls` files
- You can also input own `text_splitter` function which takes in text and returns a list of splitted text by that function. Default: LangChain's RecursiveCharacterTextSplitter
- We recommend the async method for faster accessing of memory

Example:

`memory = Memory(top_k = 5)`

`memory.add_file(file_path)`


In [17]:
# ## Run the sync version of add_file (Takes very long)

# import time

# start_time = time.time()
# memory = Memory(top_k = 1)
# memory.add_file(filepath="./react.pdf")
# memory.retrieve('What is react')

# end_time = time.time()
# print("Time taken to run the code:", end_time - start_time, "seconds")

In [27]:
# Async version of memory (Faster)
import time

start_time = time.time()
async_memory = AsyncMemory(top_k = 3)
async_memory.add_file(filepath="./react.pdf")
await async_memory.retrieve('What is react')

end_time = time.time()
print("Time taken to run the code:", end_time - start_time, "seconds")

Time taken to run the code: 6.703872919082642 seconds


In [28]:
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [29]:
# Equip this memory to the Agent and use it to answer a question
agent = AsyncAgent('Content Answerer', 'Replies to questions factually based on given memory', llm = llm_async)

In [30]:
agent.memory_bank['Document'] = async_memory

In [31]:
await async_memory.retrieve('What is ReAct?')

['.\nIn contrast to these methods, ReAct performs more than just isolated, ﬁxed reasoning, and integrates\nmodel actions and their corresponding observations into a coherent stream of inputs for the model to\nreason more accurately and tackle tasks beyond reasoning (e.g. interactive decision making)',
 'space and thought-action occurrence format, ReAct works for diverse tasks with distinct action\nspaces and reasoning needs, including but not limited to QA, fact veriﬁcation, text game, and web\nnavigation. C) Performant and robust :ReAct shows strong generalization to new task instances\nwhile learning solely from one to six in-context examples, consistently outperforming baselines with\nonly reasoning or acting across different domains. We also show in Section 3 additional beneﬁts',
 '.\nIn this work, we present ReAct , a general paradigm to combine reasoning and acting with language\nmodels for solving diverse language reasoning and decision making tasks (Figure 1). ReAct\nprompts LL

In [32]:
await agent.run('What is ReAct?')

Observation: No subtasks have been completed yet for the assigned task of explaining ReAct. However, I have gathered relevant information about ReAct, including its definition, functionality, and advantages in reasoning and decision-making tasks.
Thoughts: To complete the assigned task, I need to synthesize the gathered information into a coherent explanation of ReAct, highlighting its purpose, how it integrates reasoning and acting, and its performance across various tasks.
Subtask identified: Generate a concise and informative response that explains what ReAct is, incorporating the key points from the knowledge reference provided.
Getting LLM to perform the following task: Generate a concise and informative response that explains what ReAct is, incorporating the key points from the knowledge reference provided.
> ReAct is a paradigm designed to enhance problem-solving capabilities by integrating reasoning and acting in a structured manner. It addresses shortcomings in traditional app

['ReAct is a paradigm designed to enhance problem-solving capabilities by integrating reasoning and acting in a structured manner. It addresses shortcomings in traditional approaches by providing a more grounded, fact-driven, and trustworthy problem-solving trajectory, thanks to its access to an external knowledge base. ReAct demonstrates strong performance across diverse tasks, including question answering, fact verification, text games, and web navigation. It is particularly effective in success mode, achieving a 14% success rate compared to 6% in failure mode, with a major failure mode occurring 56% of the time. The ReAct framework is robust and adaptable, showing strong generalization to new task instances while learning from just one to six in-context examples, consistently outperforming baselines that rely solely on reasoning or acting.']

# Memory Class 2: ChromaDB Memory

## What is it?

ChromaDB Memory is a tool for storing and retrieving text-based information (like documents or tasks) using vector database technology.

- Takes in the following parameters:
    - `collection_name`: str. Compulsory. Name of the memory. Need to provide a unique name so that we can disambiguate between collections
    - `client` - Default: None. ChromaDB client to use, if any
    - `embedding_model`: Name of OpenAI's embedding_model to use with ChromaDB. Default OpenAI "text-embedding-3-small"
    - `top_k`: Number of elements to retrieve. Default: 3
    - `mapper`: Function. Maps the memory value to the embedded value. We do not need to embed the whole value, so this will serve as a way to tell us what to embed. Default: lambda x: x
     - `pre_delete`: Bool. Default: False. If set to True, delete collection with all data inside it when initialising'''

## Basic Usage

### Initialize

```python
memory = ChromaDbMemory(collection_name = "collection_name", top_k=5)
```

### Add Information

```python
memory.append(["Task: Implement login feature", "Task: Fix homepage bug"])
```

### Retrieve Information

```python
results = memory.retrieve("login feature")
```

### Remove Information

```python
memory.remove(["Task: Implement login feature"])
```

### Reset Entire ChromaDB colllection

```python
memory.reset()
```

## Async Version

For applications needing asynchronous operations, use `AsyncChromaDbMemory`:

```python
async_memory = AsyncChromaDbMemory(top_k=5)

await async_memory.append(["Async task 1", "Async task 2"])
await async_memory.retrieve("task")
```

## Tips

- Use `add_file()` to add content from text files.
- You can customize how information is stored using the `mapper` parameter when initializing.
- For better performance in large applications, consider using a persistent ChromaDB client.


In [33]:
# Sync 
memory = ChromaDbMemory(collection_name = 'new_collection', mapper = lambda x: x['text'], top_k = 2)
memory.append([{ 'text': 'This is One', 'number': 1}, {'text': 'This is Two', 'number': 2}, {'text': 'This is Three', 'number': 3},     
               {'text': 'This is Four', 'number': 4}, {'text': 'This is Five', 'number': 5}])

In [34]:
# view the chunks
memory.collection.peek(5)['metadatas']

[{'number': 1, 'text': 'This is One'},
 {'number': 3, 'text': 'This is Three'},
 {'number': 5, 'text': 'This is Five'},
 {'number': 2, 'text': 'This is Two'},
 {'number': 4, 'text': 'This is Four'}]

In [35]:
memory.retrieve("What is one")

[{'number': 1, 'text': 'This is One'}, {'number': 2, 'text': 'This is Two'}]

In [36]:
memory.append({'text': 'This is Six', 'number': 6})
memory.retrieve("What is six")

[{'number': 6, 'text': 'This is Six'}, {'number': 5, 'text': 'This is Five'}]

In [37]:
memory.remove(["This is One"])

Removing memory: This is One


In [38]:
# view the chunks
memory.collection.peek(5)['metadatas']

[{'number': 3, 'text': 'This is Three'},
 {'number': 6, 'text': 'This is Six'},
 {'number': 5, 'text': 'This is Five'},
 {'number': 2, 'text': 'This is Two'},
 {'number': 4, 'text': 'This is Four'}]

In [39]:
# Async (if we are using a sync client for ChromaDb, it will use the sync functions)
memory = AsyncChromaDbMemory(collection_name = 'new_collection_async', mapper = lambda x: x['text'], top_k = 2)

In [40]:
await memory.append([{ 'text': 'This is One', 'number': 1}, {'text': 'This is Two', 'number': 2}, {'text': 'This is Three', 'number': 3},     
     {'text': 'This is Four', 'number': 4}, {'text': 'This is Five', 'number': 5}])

In [41]:
# view the chunks
memory.collection.peek(5)['metadatas']

[{'number': 4, 'text': 'This is Four'},
 {'number': 3, 'text': 'This is Three'},
 {'number': 1, 'text': 'This is One'},
 {'number': 2, 'text': 'This is Two'},
 {'number': 5, 'text': 'This is Five'}]

In [42]:
await memory.retrieve("What is one")

[{'number': 1, 'text': 'This is One'}, {'number': 2, 'text': 'This is Two'}]

In [43]:
await memory.reset()

In [44]:
memory = AsyncChromaDbMemory(collection_name = "file_collection", top_k = 1)
await memory.add_file(filepath="./react.pdf")

In [45]:
await memory.retrieve("What is react")

[{'content': '.\nIn contrast to these methods, ReAct performs more than just isolated, ﬁxed reasoning, and integrates\nmodel actions and their corresponding observations into a coherent stream of inputs for the model to\nreason more accurately and tackle tasks beyond reasoning (e.g. interactive decision making)',
  'filepath': './react.pdf'}]

## For even faster ChromaDB experience, use an Async Client

In [46]:
# Chroma db with external Async Client
# For below experiment you need to follow following steps to run chromadb server
    # git clone https://github.com/chroma-core/chroma
    # cd chroma
    # docker-compose up -d --build

In [47]:
# import chromadb
# chroma_client = await chromadb.AsyncHttpClient(host="localhost", port=8000)
# memory = AsyncChromaDbMemory(collection_name = "1234_collection",top_k = 1, client = chroma_client)
# await memory.add_file(filepath="./react.pdf")

In [39]:
# await memory.retrieve("What is react")